<a href="https://colab.research.google.com/github/krasomil/CodeMonkeyApplication/blob/master/LB_score_32%F0%9F%8F%86%E2%98%A2%EF%B8%8FRadon_Milosz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>☢️Radon</font>](https://www.kaggle.com/c/16oct23jh-radon/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [1]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Mounted at /content/drive


In [2]:
!pip install --upgrade --force-reinstall --no-deps kaggle >> log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >>log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 16oct23jh-radon # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 16oct23jh-radon
100% 538k/538k [00:00<00:00, 156MB/s]
Using competition: 16oct23jh-radon
  teamId  teamName               submissionDate       score     
--------  ---------------------  -------------------  --------  
11109033  Team 5                 2023-10-27 15:06:12  23.21768  
11120945  Team 4                 2023-10-28 02:29:14  30.99993  
11088583  Ilian Torres           2023-10-28 15:53:07  30.99993  
11114352  milosz.krasowski       2023-10-28 19:46:31  32.05898  
11136754  JW_Rogers              2023-10-28 17:07:42  33.21113  
11126746  Kate Kennelly          2023-10-28 03:33:49  33.45593  
11121828  Ira Dhalawong          2023-10-28 19:37:25  34.79255  
11125603  Alec Wayne             2023-10-28 14:06:53  35.63957  
11132802  Bennett Blitz          2023-10-28 08:24:16  37.31084  
11124289  Marko Zlatic           2023-10-28 03:14:43  40.92389  
11123684  Claribel Gonell        2023-10-25 2

In [3]:
!pip install -U tfds-nightly tensorflow_addons tensorflow keras==2.12.0 uszipcode >> log # downgrade keras to bypass tf_utils import error

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.


In [4]:

%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, tensorflow_addons as tfa, tensorflow as tf, tensorflow.keras as keras, os
from keras.layers import Flatten, Dense
os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'; # allows seeding RNG on GPU
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 3.11 s, sys: 425 ms, total: 3.54 s
Wall time: 4.05 s


In [5]:
df_raw = pd.read_csv('XY_radon.csv'); df_raw

,Uppm,adjwt,basement,cntyfips,county,dupflag,floor,lat,lon,pcterr,...,stfips,stopdt,stoptm,stratum,typebldg,wave,windoor,zip,zipflag,Y
0,1.80,54.97,Y,59,MORTON,0,1,46.66,-101.39,7.76,...,38,32206,1230,2,2,1,NaN,58554,0,NaN
1,1.65,499.34,N,85,KOSCIUSKO,0,1,40.85,-86.22,55.02,...,18,11497,1430,3,1,32,NaN,46580,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,0.44,394.07,Y,3,ANOKA,0,0,44.91,-92.86,9.40,...,27,32110,1500,2,1,4,NaN,55303,0,8.6
12572,2.71,157.82,Y,15,MOHAVE,0,0,36.01,-113.21,14.46,...,4,11496,1330,1,1,38,NaN,86403,0,1.9


In [6]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, code, documentation, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

In [7]:
#Import packages
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
from tensorflow.keras import callbacks
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.layers import Flatten, Dense, Dropout
import keras
from keras import backend as K
import random
import os
from sklearn.cluster import KMeans

In [8]:
#Ensure repeatability
np.random.seed(13)
keras.utils.set_random_seed(13)
tf.random.set_seed(0)
# tf.config.experimental.enable_op_determinism()

<font color=green><h3><b>$\alpha$. Split observations into train and test sets</b><h3>

In [9]:
# url = 'https://www.epa.gov/sites/default/files/2020-03/radon_zones-spreadsheet.xls'
# storage_options = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"}
# zones = pd.read_excel(url, storage_options=storage_options)

# zones['County,State'] = zones['County,State'].apply(lambda x: str(x).upper().replace('.', '').replace(' ', '').replace('STCROIX','ST.CROIX').replace('SAGINAW', 'SAGINAWCHIPPEWA'))

# zones = zones[['County,State', 'Zone']].set_index(zones['County,State']).drop(['County,State'], axis=1).dropna().to_dict()

# df_raw['zone'] = df_raw.apply(lambda x: zones['Zone'][str(x['county']) + "," + str(x['state2'])] if (str(x['county']) + "," + str(x['state2']) in zones['Zone'].keys()) else np.nan, axis=1)

# #Replace 3 with 1. Because 3 is the lowest radiation level in the original data
# df_raw.loc[df_raw['zone']==3, 'zone'] = 93
# df_raw.loc[df_raw['zone']==1, 'zone'] = 3
# df_raw.loc[df_raw['zone']==93, 'zone'] = 1

In [10]:
# Haversine distance formula can be found here: https://www.geeksforgeeks.org/haversine-formula-to-find-distance-between-two-points-on-a-sphere/#
def haversine(lat1, lon1, lat2, lon2):

    dLat = (lat2 - lat1) * np.pi / 180.0
    dLon = (lon2 - lon1) * np.pi / 180.0

    lat1 = lat1 * np.pi / 180.0
    lat2 = lat2 * np.pi / 180.0

    a = (np.sin(dLat / 2) ** 2 + np.sin(dLon / 2) ** 2 * np.cos(lat1) * np.cos(lat2))

    rad = 6371  # radius of the Earth

    c = 2 * np.arcsin(np.sqrt(a))

    return rad * c

In [11]:
# link to counties csv with coordinates
zip_file = 'https://simplemaps.com/static/data/us-counties/1.73/basic/simplemaps_uscounties_basicv1.73.zip'
# below code found at: https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url
import zipfile, requests, io

r = requests.get(zip_file)
with zipfile.ZipFile(io.BytesIO(r.content)) as zip_ref:
    zip_ref.extractall(os.getcwd())


In [12]:
county_coords = pd.read_csv(os.path.join(os.getcwd(), 'uscounties.csv'))[['county', 'state_id', 'lat', 'lng']]
county_coords['join_key'] = county_coords[['county', 'state_id']].apply(lambda row: row['county'] + ', ' + row['state_id'], axis=1)
coord_county_names = county_coords['join_key'].values.tolist()

In [13]:
url = 'https://www.epa.gov/sites/default/files/2020-03/radon_zones-spreadsheet.xls'
storage_options = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"}
zones = pd.read_excel(url, storage_options=storage_options).query('`COUNTY LABEL`!= "no data"')

radon_counties = zones['County,State'].values.tolist()
df_raw['County,State'] = df_raw[['county', 'state']].apply(lambda row: str(row['county']).title() + ', ' + row['state'], axis=1)


In [14]:
# df_raw['distance_to_zone'] = df_raw[['County,State', 'lat', 'lon']].apply(lambda row: haversine(county_coords[county_coords['join_key'] == row['County,State']]['lat'].values.tolist()[0], county_coords[county_coords['join_key'] == row['County,State']]['lng'].values.tolist()[0], row['lat'], row['lon']) if row['County,State'] in coord_county_names and row['County,State'] in radon_counties else np.nan, axis=1)

In [15]:
# Create an array of coordinates for 'County,State' from county_coords
county_coords_dict = dict(zip(county_coords['join_key'], zip(county_coords['lat'], county_coords['lng'])))
county_coords_array = np.array([county_coords_dict.get(county_state, (np.nan, np.nan)) for county_state in df_raw['County,State']])

# Calculate haversine distances for all rows
haversine_distances = np.array([haversine(lat1, lon1, lat2, lon2) for lat1, lon1, (lat2, lon2) in zip(county_coords_array[:,0], county_coords_array[:,1], zip(df_raw['lat'], df_raw['lon']))])

# Apply conditions and assign distances or NaN
mask = np.isin(df_raw['County,State'], coord_county_names) & np.isin(df_raw['County,State'], radon_counties)
df_raw['distance_to_zone'] = np.where(mask, haversine_distances, np.nan)

In [16]:
# kmeans = KMeans(n_clusters=8, n_init='auto', random_state=0)
# resulting_clusters = kmeans.fit(df_raw[['lon', 'lat']])
# cluster_centers = pd.DataFrame(resulting_clusters.cluster_centers_, columns=['lon', 'lat']).reset_index(names='cluster_id')

# # calculate cluster ids
# cluster_ids = pd.DataFrame(resulting_clusters.labels_, columns=['cluster_id']).reset_index(names='join_field')
# df_raw = df_raw.reset_index(names='join_field').merge(cluster_ids, on='join_field').drop(columns='join_field', axis=1)

# # calculate distance from center cluster and weighted inverse
# df_raw['cluster_center_distance'] = df_raw[['cluster_id', 'lon', 'lat']].apply(lambda row: haversine(cluster_centers[cluster_centers['cluster_id'] == row['cluster_id']]['lat'].values.tolist()[0], cluster_centers[cluster_centers['cluster_id'] == row['cluster_id']]['lon'].values.tolist()[0], row['lat'], row['lon']), axis=1)
# df_raw['cluster_center_weighted_inverse'] = 1 / df_raw['cluster_center_distance']

# # calculate distance from center point and weighted inverse
# median_lon, median_lat = np.median(df_raw['lon']), np.median(df_raw['lat'])
# df_raw['dist_from_median_center'] = df_raw[['lon', 'lat']].apply(lambda row: haversine(median_lat, median_lon, row['lat'], row['lon']), axis=1)
# df_raw['median_center_weighted_inverse'] = 1 / df_raw['dist_from_median_center']

In [17]:
#Build a pipeline
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, PolynomialFeatures, PowerTransformer, OrdinalEncoder
from sklearn.impute import SimpleImputer


num_feat = ['Uppm','adjwt','cntyfips','dupflag','floor', 'lat', 'lon', 'pcterr','region', 'rep', 'room', 'startdt', 'starttm','stfips', 'stopdt',
       'stoptm', 'stratum', 'typebldg','wave','distance_to_zone']#,'zone' ,'cluster_center_distance','cluster_center_weighted_inverse','dist_from_median_center','median_center_weighted_inverse'
hotencode_feat = ['state2','basement','zip'] #, 'cluster_id'


num_pipeline = Pipeline([
    ('scaler', PowerTransformer())
    ])
# categories = [['46','85','63','58','64','18','23','17','14','56','19','54','65','55','47','15','48','86','20','11','16','27','21','26','49','12','10','25','13','24','59','70']]
onehot_pipeline = Pipeline([
    ("encoder", OneHotEncoder(sparse_output=False,  handle_unknown='infrequent_if_exist'))
    ])
dataset_transformer = ColumnTransformer(
    transformers=[
        ('scaler', num_pipeline, num_feat),
        ('onehot', onehot_pipeline, hotencode_feat),
    ],
    remainder='passthrough'
)

pipe = Pipeline([
                  ('transformer',dataset_transformer),
                  ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                  # ('scaler', StandardScaler()),
                  # ('polynomial', PolynomialFeatures(2)),
                  # ('pca', PCA(250)),
                ])

# tX_transformed = pipe.fit_transform(tX)
# vX_transformed = pipe.fit_transform(vX)

In [18]:
#Drop the unnecessary data
#drop county because it contains missing values and is approximated by cntyfips
#drop state because state2 contains better data. state has a weird value 'R5' which doesn't exist in state2
#drop zipflag. It's not needed
df = df_raw.drop(columns=['windoor','county', 'state', 'zipflag','County,State'], axis=1)
#Convert 'N' to 0 and 'Y' to 1 for basement
df.loc[df['basement']== 'N', 'basement'] = 0
df.loc[df['basement']== 'Y', 'basement'] = 1
df['basement'] = df['basement'].apply(str)
#Replace one zip which equals -1 with 58646. I checked the zipcode in google maps by inserting the latitude and longitude for that record
df.loc[df['zip']==-1, 'zip'] = 58646
#Keep only 2 digits from the zip code
df['zip'] = df['zip'].apply(str)
df['zip'] = df['zip'].str[:2]
df['zip'] = df['zip'].apply(str)
#split into tX and tY
tX, tY = df.drop('Y', axis=1), df.Y
#transform data with the pipeline
tX_transformed = pd.DataFrame(pipe.fit_transform(tX))
df = pd.concat([tX_transformed, tY], axis=1)
#Split into train and validation datasets
vX_transformed = df.query('Y!=Y').drop('Y', axis=1)    # slice a test sample
tXY = df.query('Y==Y')                     # slice training sample
tX_transformed, tY = tXY.drop('Y', axis=1), tXY.Y.astype(int)      # split into training I/O
# print(tY.tolist()[:50])                    # train outputs

<font color=green><h3><b>$\beta$. Build and train a model</b><h3>

In [19]:
# trainX, testX, trainY, testY = train_test_split(tX, tY, train_size=0.7, test_size=0.3, shuffle = True, random_state=13)

In [20]:
# #TUNING
# !pip install keras-tuner --upgrade
# import keras_tuner

# batch_size = 50

# # Reduce learning rate on plateau
# callbacks = [
#     # tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=0),
#     tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.3, verbose=0)
# ]


# # number_hidden_layers_options = [1,2,3,4]
# # neurons_per_layer_options = np.arange(1, 1000, step = 10)
# # learning_rate_options = [x for x in np.geomspace(0.1, 1e-7, num=7)]
# # dropout_rate_options = [0.0, 0.1, 0.15, 0.2]
# # l2_weight_options = [0.0, 0.001, 0.0001, 0.00001, 0.000001]
# # activation_func_options = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'elu', 'selu', tf.keras.layers.LeakyReLU(alpha=0.05)]
# # optimization_func_options =  ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
# # init_func_options = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform',keras.initializers.RandomNormal(seed=0)]

# def build_model(hp):
#     # Tune whether to add Dropout layers
#     # hp_add_dropout = hp.Choice('add_dropout', values=[True, False])

#     # Tune whether to add BatchNormalization layers
#     # hp_add_bn = hp.Choice('add_bn', values=[True, False])

#     model = keras.Sequential()
#     model.add(keras.layers.Flatten(input_shape=[tX_transformed.shape[1]]))

#     # if hp_add_dropout:
#     #     # Tune the dropout rate from [0.1, 0.2, 0.3]
#     #     model.add(keras.layers.Dropout(rate=hp.Choice('dropout_rate_0', values=[0.1, 0.2, 0.0,0.3], default=0.1)))

#     # if hp_add_bn:
#     #     model.add(keras.layers.BatchNormalization())

#     # Tune the L1 and L2 rates for the regularizer from [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
#     hp_l1_lr = hp.Choice('l1_lr', values=[0.0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0], default=1e-6) #0.0#
#     hp_l2_lr = hp.Choice('l2_lr', values=[0.0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0], default=1e-5) #1e-05 #
#     hp_bias_lr = hp.Choice('bias_lr', values=[0.0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0], default=1e-4) #0.0001 #

#     # Tune activation function + initializer
#     a_act_opts = {
#               0: ('softmax'),
#               1: ('softplus'),
#               2: ('softsign'),
#               3: ('relu'),
#               4: ('tanh'),
#               5: ('sigmoid'),
#               6: ('hard_sigmoid'),
#               7: ('linear'),
#               8: ('elu'),
#               9: ('selu'),
#               10: (tf.keras.layers.LeakyReLU(alpha=0.05)),
#               11: ('gelu'),
#             }
#     b_init_options = {
#               # 0: ('uniform'),
#               # 1: ('lecun_uniform'),
#               # 2: ('normal'),
#               # 3: ('zero'),
#               # 4: ('glorot_normal'),
#               # 5: ('glorot_uniform'),
#               # 6: ('he_normal'),
#               0: (tf.keras.initializers.HeNormal(seed=13)),
#               1: (keras.initializers.RandomNormal(seed=0))
#             }
#     hp_a = hp.Int('activation', 0, 10, default=10)
#     hp_b = hp.Int('init', 0, 1, default=0)
#     n_layers = hp.Int('num_layers', 1, 6, default=3)
#     # Tune the number of layers
#     for i in range(n_layers):

#         # Tune the number of neurons per layer + regularizers
#         model.add(keras.layers.Dense(units=hp.Int(f"units_{i}",
#                                                   min_value=10,
#                                                   max_value=125,
#                                                   step=1,
#                                                   default=80),
#                                activation=a_act_opts[hp_a],
#                                kernel_initializer=b_init_options[hp_b],
#                                kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
#                                bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),)
#                  )

#         # if hp_add_dropout:
#         #     # Tune the dropout rate from [0.1, 0.2, 0.3]
#         #     model.add(keras.layers.Dropout(rate=hp.Choice(f"dropout_rate_{i + 1}", values=[0.1, 0.2, 0.3, 0.4], default=0.1)))

#         # if hp_add_bn:
#         #     model.add(keras.layers.BatchNormalization())

#     model.add(keras.layers.Dense(1, kernel_initializer=b_init_options[hp_b], name='output'))

#     # Tune the learning rate for the optimizer from [0.01, 0.001, or 0.0001]
#     # hp_learning_rate = hp.Choice('learning_rate', values=[x for x in np.geomspace(0.1, 1e-7, num=7)], default=0.01)
#     hp_learning_rate = hp.Choice('learning_rate', values=[0.05,0.02,0.01,0.005])
#     # Tune optimizer beta 1 and beta 2 from [0.7, 0.9, 0.99, 0.999]
#     hp_beta1 = hp.Choice('beta_1', values=[0.7, 0.9, 0.99, 0.999]) #0.9 #
#     hp_beta2 = hp.Choice('beta_2', values=[0.7, 0.9, 0.99, 0.999]) # 0.9 #
#     # LRSchedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=hp_learning_rate, decay_steps=10000, decay_rate=0.9)
#     c_opt_options = {
#               0: (keras.optimizers.Nadam(learning_rate= hp_learning_rate, beta_1=hp_beta1, beta_2=hp_beta2)),
#               1: (keras.optimizers.SGD(learning_rate=hp_learning_rate)),
#               2: (keras.optimizers.RMSprop(learning_rate=hp_learning_rate)),
#               3: (keras.optimizers.Adagrad(learning_rate=hp_learning_rate)),
#               4: (keras.optimizers.Adadelta(learning_rate=hp_learning_rate)),
#               5: (keras.optimizers.Adam(learning_rate=hp_learning_rate)),
#               6: (keras.optimizers.Adamax(learning_rate=hp_learning_rate))
#             }
#     hp_c = hp.Int('optimizer', 0, 6, default=0)
#     model.compile(optimizer=c_opt_options[hp_c],
#                   loss='mse',
#                   metrics=['mse'])
#     return model

# # Exponential Decay for learning rate

# tuner = keras_tuner.RandomSearch(
#     build_model,
#     objective= keras_tuner.Objective("val_mse", direction="min"),
#     max_trials=50,  # The maximum number of hyperparameter combinations to try
#     executions_per_trial=1, # The number of times to train the model for each combination of hyperparameters
#     # directory='dnn', # The directory where to save the results of the search
#     project_name='trial1', # name the directory where to save the results.
# )

# tuner.search(tX_transformed, tY, epochs=30, validation_split=0.3,
#              batch_size=batch_size,
#              callbacks=callbacks,
#             )

# best_model = tuner.get_best_models()[0]

In [21]:
# best_model.summary()

In [22]:
# tuner.results_summary()

In [23]:
#THIS IDEA WAS USED BY TEAM 7 IN ASSIGNMENT 5. KUDOS TO TEAM 7.
# Workaround to try to immediately return when all training epochs complete
# Otherwise we'd see delays of 20s to a minute after training

class TrainingComplete(Callback):
    def on_train_end(self, logs=None):
        print("Training has ended.")
        self.end_time = time.time()
        raise('End quick')

In [24]:
# Hyperparameters:
# l1_lr: 1e-05
# l2_lr: 1e-05
# bias_lr: 0.1
# activation: 2
# init: 0
# num_layers: 6
# units_0: 111
# units_1: 98
# units_2: 65
# learning_rate: 0.05
# beta_1: 0.7
# beta_2: 0.99
# optimizer: 6
# units_3: 26
# units_4: 111
# units_5: 91
# Score: 20.706668853759766

#     c_opt_options = {
#               0: (keras.optimizers.Nadam(learning_rate= hp_learning_rate, beta_1=hp_beta1, beta_2=hp_beta2)),
#               1: (keras.optimizers.SGD(learning_rate=hp_learning_rate)),
#               2: (keras.optimizers.RMSprop(learning_rate=hp_learning_rate)),
#               3: (keras.optimizers.Adagrad(learning_rate=hp_learning_rate)),
#               4: (keras.optimizers.Adadelta(learning_rate=hp_learning_rate)),
#               5: (keras.optimizers.Adam(learning_rate=hp_learning_rate)),
#               6: (keras.optimizers.Adamax(learning_rate=hp_learning_rate))

#    a_act_opts = {
#               0: ('softmax'),
#               1: ('softplus'),
#               2: ('softsign'),
#               3: ('relu'),
#               4: ('tanh'),
#               5: ('sigmoid'),
#               6: ('hard_sigmoid'),
#               7: ('linear'),
#               8: ('elu'),
#               9: ('selu'),
#               10: (tf.keras.layers.LeakyReLU(alpha=0.05))
#             }
#     b_init_options = {
#               0: ('uniform'),
#               1: ('lecun_uniform'),
#               2: ('normal'),
#               3: ('zero'),
#               4: ('glorot_normal'),
#               5: ('glorot_uniform'),
#               6: ('he_normal'),
#               7: ('he_uniform'),
#               8: (keras.initializers.RandomNormal(seed=0))



# Init = keras.initializers.RandomNormal(seed=13)
Init = tf.keras.initializers.HeNormal(seed=13)
activation = 'softsign'
hp_learning_rate = 0.01
hp_beta1 = 0.7
hp_beta2 = 0.99
hp_l1_lr=  1e-05
hp_l2_lr= 1e-05
hp_bias_lr = 0.1
leakyReluAlpha = 0.05
LRSchedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=hp_learning_rate, decay_steps=10000, decay_rate=0.90)
opt = keras.optimizers.Adamax(learning_rate=hp_learning_rate)

m = keras.Sequential()
m.add(keras.layers.Flatten(input_shape=[tX_transformed.shape[1]]))
# m.add(keras.layers.Dropout(0))
# earlyStopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=0),
# reducedLR = tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.4, verbose=0)

#Hidden1
m.add(keras.layers.Dense(units=111,
                        activation=activation,
                        kernel_initializer=Init,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
                        bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# m.add(keras.layers.Dropout(0.2))

#Hidden2
m.add(keras.layers.Dense(units=98,
                        activation=activation,
                        kernel_initializer=Init,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
                        bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# m.add(keras.layers.Dropout(0.4))

#Hidden3
m.add(keras.layers.Dense(units=65,
                        activation=activation,
                        kernel_initializer=Init,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
                        bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# m.add(keras.layers.Dropout(0.2))

#Hidden4
m.add(keras.layers.Dense(units=26,
                        activation=activation,
                        kernel_initializer=Init,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
                        bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# m.add(keras.layers.Dropout(0.2))

#Hidden5
m.add(keras.layers.Dense(units=111,
                        activation=activation,
                        kernel_initializer=Init,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
                        bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# m.add(keras.layers.Dropout(0.2))

#Hidden6
m.add(keras.layers.Dense(units=91,
                        activation=activation,
                        kernel_initializer=Init,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
                        bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# m.add(keras.layers.Dropout(0.2))


#Output Layer
m.add(keras.layers.Dense(1, kernel_initializer=Init, name='output'))
m.compile(optimizer=opt,loss='mse',metrics=['mse'])
m.summary()
training_complete = TrainingComplete()

# # tf. config. list_physical_devices()
with tf.device('/device:GPU:0'):
  try:
    m.fit(tX_transformed, tY, epochs=43, batch_size=50, verbose=10, callbacks=[training_complete]) #callbacks=[learningRateCallBack, training_complete] , validation_data=(sX1_transformed, sY1) reducedLR  , validation_split=0.3
  except:
    print('Quick ending')

# m.fit(tX_transformed, tY, epochs=43, batch_size=50,validation_split=0.3)

#EPOCHS 43 is best but take too much time 42 is much faster


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 67)                0         
                                                                 
 dense (Dense)               (None, 111)               7548      
                                                                 
 dense_1 (Dense)             (None, 98)                10976     
                                                                 
 dense_2 (Dense)             (None, 65)                6435      
                                                                 
 dense_3 (Dense)             (None, 26)                1716      
                                                                 
 dense_4 (Dense)             (None, 111)               2997      
                                                                 
 dense_5 (Dense)             (None, 91)                1

In [25]:
from sklearn.metrics import mean_squared_error
predictionTest = m.predict(tX_transformed)
#clip predictions
predictionTest = np.maximum(predictionTest, 0)
print(f"MSE score: {mean_squared_error(tY, predictionTest)}")

197/197 [==============================] - 1s 2ms/step
MSE score: 10.074623674867034


In [26]:
# from sklearn.metrics import mean_squared_error
# predictionTest = m.predict(tX_transformed)
# print(f"MSE score: {mean_squared_error(tY, predictionTest)}")

The model generates a baseline submission CSV file, see Colab folder (🗀 on the left), which you candownload and submit to Kaggle.

<font color=green><h3><b>$\gamma$. Make predictions</b><h3>

In [27]:
#Create predictions and clip lower values to 0
pY = pd.DataFrame(np.maximum(m.predict(vX_transformed), 0), index=np.arange(len(vX_transformed))+1, columns=['y'])
ToCSV(pY.round(1), '☢️Milosz submission6')

197/197 [==============================] - 0s 2ms/step


<font color=green><h3><b>$\delta$. Documentation</b></h3></font>

<font color=green><h4><b>Task 1. Explain Decisions in Preprocessing Pipeline</b></h4></font>

<font color=green>
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.</font>

<font color=green>

1. Why did you choose these elements? (Something in EDA, prior experience,...? Note: EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

<font color=red><b>Your answer here.</b></font>

<font color=green><h4><b>Task 2. Explain Decisions in Modeling Pipeline</b></h4></font>

<font color=green>
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

<font color=red><b>Your answer here.</b></font>

<font color=green><h3><b>$\epsilon$. References</b></h3></font>

<font color=red><b>Your answer here.</b></font>

<font color=green>
Cite your sources to help your peers learn from these (and to avoid plagiarism claims). At the least, HOML textbook should be cited. Use Google Scholar to draw APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources.

1. ...
1. ...

<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 49 sec


In [ ]:
# # Trial 10 summary
# # Hyperparameters:
# # l1_lr: 1e-06
# # l2_lr: 0.01
# # bias_lr: 0.1
# # activation: 8
# # init: 0
# # num_layers: 4
# # units_0: 8
# # units_1: 93
# # units_2: 86
# # learning_rate: 0.01
# # beta_1: 0.999
# # beta_2: 0.9
# # optimizer: 5
# # units_3: 80
# # units_4: 92
# # Score: 20.267539978027344


# #     c_opt_options = {
# #               0: (keras.optimizers.Nadam(learning_rate= hp_learning_rate, beta_1=hp_beta1, beta_2=hp_beta2)),
# #               1: (keras.optimizers.SGD(learning_rate=hp_learning_rate)),
# #               2: (keras.optimizers.RMSprop(learning_rate=hp_learning_rate)),
# #               3: (keras.optimizers.Adagrad(learning_rate=hp_learning_rate)),
# #               4: (keras.optimizers.Adadelta(learning_rate=hp_learning_rate)),
# #               5: (keras.optimizers.Adam(learning_rate=hp_learning_rate)),
# #               6: (keras.optimizers.Adamax(learning_rate=hp_learning_rate))

# #    a_act_opts = {
# #               0: ('softmax'),
# #               1: ('softplus'),
# #               2: ('softsign'),
# #               3: ('relu'),
# #               4: ('tanh'),
# #               5: ('sigmoid'),
# #               6: ('hard_sigmoid'),
# #               7: ('linear'),
# #               8: ('elu'),
# #               9: ('selu'),
# #               10: (tf.keras.layers.LeakyReLU(alpha=0.05))
# #             }
# #     b_init_options = {
# #               0: ('uniform'),
# #               1: ('lecun_uniform'),
# #               2: ('normal'),
# #               3: ('zero'),
# #               4: ('glorot_normal'),
# #               5: ('glorot_uniform'),
# #               6: ('he_normal'),
# #               7: ('he_uniform'),
# #               8: (keras.initializers.RandomNormal(seed=0))



# # Init = keras.initializers.RandomNormal(seed=13)
# Init = tf.keras.initializers.HeNormal(seed=13)
# activation = 'elu'
# hp_learning_rate = 0.01
# hp_beta1 = 0.999
# hp_beta2 = 0.9
# hp_l1_lr=  1e-06
# hp_l2_lr= 0.01
# hp_bias_lr = 0.1
# leakyReluAlpha = 0.05
# LRSchedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=hp_learning_rate, decay_steps=10000, decay_rate=0.90)
# opt = keras.optimizers.Adam(learning_rate=hp_learning_rate)

# m = keras.Sequential()
# m.add(keras.layers.Flatten(input_shape=[tX_transformed.shape[1]]))
# # m.add(keras.layers.Dropout(0))
# # earlyStopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=0),
# # reducedLR = tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.4, verbose=0)

# #Hidden1
# m.add(keras.layers.Dense(units=30,
#                         activation=activation,
#                         kernel_initializer=Init,
#                         kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
#                         bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# # m.add(keras.layers.Dropout(0.2))

# #Hidden2
# m.add(keras.layers.Dense(units=93,
#                         activation=activation,
#                         kernel_initializer=Init,
#                         kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
#                         bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# # m.add(keras.layers.Dropout(0.4))

# #Hidden3
# m.add(keras.layers.Dense(units=86,
#                         activation=activation,
#                         kernel_initializer=Init,
#                         kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
#                         bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# # m.add(keras.layers.Dropout(0.2))

# #Hidden4
# m.add(keras.layers.Dense(units=80,
#                         activation=activation,
#                         kernel_initializer=Init,
#                         kernel_regularizer=tf.keras.regularizers.L1L2(l1=hp_l1_lr, l2=hp_l2_lr),
#                         bias_regularizer=tf.keras.regularizers.L2(l2=hp_bias_lr),))
# # m.add(keras.layers.Dropout(0.2))

# #Output Layer
# m.add(keras.layers.Dense(1, kernel_initializer=Init, name='output'))
# m.compile(optimizer=opt,loss='mse',metrics=['mse'])
# m.summary()
# training_complete = TrainingComplete()

# # # tf. config. list_physical_devices()
# with tf.device('/device:GPU:0'):
#   try:
#     m.fit(tX_transformed, tY, epochs=37, batch_size=50, verbose=10, callbacks=[training_complete]) #callbacks=[learningRateCallBack, training_complete] , validation_data=(sX1_transformed, sY1) reducedLR  , validation_split=0.3
#   except:
#     print('Quick ending')

# # m.fit(tX_transformed, tY, epochs=30, batch_size=50,validation_split=0.3)


In [ ]:
# from sklearn.metrics import mean_squared_error
# predictionTest = m.predict(tX_transformed)
# print(f"MSE score: {mean_squared_error(tY, predictionTest)}")

# **Starter Ideas**

1. Try different DNN architectures and tuning of hyperparameters
1. Try converting locations to distances to the key Radon sources (which you might need to discover).
1. Try clustering categorical variables by their relation to Radon levels
1. Try replacing categorical values with their level frequencies or other encodings
1. Try scaling features linearly or nonlinearly
1. Try embedding **textual** values (eg. US States names) with pre-trained SBERT-like models. This injects some additional information from Wikipedia (or whichever corpora were used for model training).
1. Do EDA and understand the variables and their relation to the output. [Example 1](https://docs.pymc.io/en/v3.11.4/pymc-examples/examples/case_studies/multilevel_modeling.html), [Example 2](https://www.tensorflow.org/probability/examples/Multilevel_Modeling_Primer)

<hr>
<font color=black>
    <details><summary><font color=carnelian>▶ </font>Clustering categorical variables <b></b>.</summary>

  1. When we represent categorical variables as dummies, we may be losing important multivariate information. For example, say we use weekdays to predict the number of hours a person works. We could convert weekdays to 6 features (one is dropped due to collinearity). This requires 6 coefficients (degrees of freedom or sources of uncertainty). Essentially, we have an overparameterized model, whereas all we really need is two clusters of categorical values - weekends (Sat/Sun) and non-weekends (M/T/W/Th/F). In general, the model overparameterized model will do worse due to higher variance of the model output (resulting from the overfit and higher flexibility).

  1. Here is another example from the NLP domain, where each word is a feature (or dimension). While morphological variants of a word (eg. run, running, runner, ran, runs, ...) have lower frequency, we cluster them into the same lemma "run", assuming only a small loss of semantic information. We hope that the gain in building a better distribution estimate for "run" is greater than the loss of semantic and lexical information.
        </details>
    <details><summary><font color=carnelian>▶ </font>Distance to Radon source<b></b>.</summary>

If you can determine where Radon is most active (i.e. the source), then you might be able to compute the distance to the source. Ordinarily, we expect lower radiation for greater distance from the source (assuming uniform distribution of underground rivers, geology, rains/winds and other weather conditions affecting distribution of radon, etc.). You could also use categorical features in (e.g. US State, region, etc.), but these might perform better when clustered (again). Distance to the source is a real-valued feature, which does not require clustering.
        </details>
</font>

In [ ]:
# tf.random.set_seed(0)   # always seed your experiments
# Init = keras.initializers.RandomNormal(seed=0)

# m = keras.models.Sequential([
#   Flatten(input_shape=[tX.shape[1]]),
#   Dense(5, activation="relu", kernel_initializer=Init),
#   Dense(5, activation="relu", kernel_initializer=Init),
#   Dense(5, activation="relu", kernel_initializer=Init),
#   Dense(1, kernel_initializer=Init)])
# m.summary()
# m.compile(loss="mse", optimizer="adam", metrics=['mse'])
# hist = m.fit(x=tX, y=tY, batch_size=32, epochs=5, validation_split=0.3)

In [ ]:
# Model: "sequential"
# _________________________________________________________________
#  Layer (type)                Output Shape              Param #
# =================================================================
#  flatten (Flatten)           (None, 67)                0

#  dense (Dense)               (None, 4)                 272

#  dense_1 (Dense)             (None, 86)                430

#  dense_2 (Dense)             (None, 87)                7569

#  output (Dense)              (None, 1)                 88

# =================================================================
# Total params: 8,359
# Trainable params: 8,359
# Non-trainable params: 0
# _________________________________________________________________



# Trial 11 summary
# Hyperparameters:
# l1_lr: 1e-05
# l2_lr: 0.01
# bias_lr: 1e-05
# activation: 8
# init: 1
# num_layers: 3
# units_0: 4
# units_1: 86
# units_2: 87
# learning_rate: 0.05
# beta_1: 0.9
# beta_2: 0.9
# optimizer: 2
# units_3: 45
# units_4: 23
# Score: 17.085132598876953

# Trial 10 summary
# Hyperparameters:
# l1_lr: 1e-06
# l2_lr: 0.01
# bias_lr: 0.1
# activation: 8
# init: 0
# num_layers: 4
# units_0: 8
# units_1: 93
# units_2: 86
# learning_rate: 0.01
# beta_1: 0.999
# beta_2: 0.9
# optimizer: 5
# units_3: 80
# units_4: 92
# Score: 20.267539978027344

# Trial 13 summary
# Hyperparameters:
# l1_lr: 0.0001
# l2_lr: 0.001
# bias_lr: 0.0001
# activation: 5
# init: 1
# num_layers: 3
# units_0: 84
# units_1: 93
# units_2: 35
# learning_rate: 0.1
# beta_1: 0.99
# beta_2: 0.9
# optimizer: 1
# units_3: 10
# units_4: 37
# Score: 32.629981994628906

# Trial 04 summary
# Hyperparameters:
# l1_lr: 1e-05
# l2_lr: 0.0
# bias_lr: 0.0001
# activation: 3
# init: 0
# num_layers: 3
# units_0: 88
# units_1: 75
# units_2: 43
# learning_rate: 0.1
# beta_1: 0.99
# beta_2: 0.7
# optimizer: 3
# units_3: 98
# units_4: 8
# Score: 33.90720748901367



In [ ]:
# _________________________________________________________________
#  Layer (type)                Output Shape              Param #
# =================================================================
#  flatten (Flatten)           (None, 67)                0

#  dense (Dense)               (None, 111)               7548

#  dense_1 (Dense)             (None, 98)                10976

#  dense_2 (Dense)             (None, 65)                6435

#  dense_3 (Dense)             (None, 26)                1716

#  dense_4 (Dense)             (None, 111)               2997

#  dense_5 (Dense)             (None, 91)                10192

#  output (Dense)              (None, 1)                 92

# =================================================================
# Total params: 39,956
# Trainable params: 39,956
# Non-trainable params: 0
# _________________________________________________________________

# Results summary
# Results in ./trial1
# Showing 10 best trials
# Objective(name="val_mse", direction="min")

# Trial 20 summary
# Hyperparameters:
# l1_lr: 1e-05
# l2_lr: 1e-05
# bias_lr: 0.1
# activation: 2
# init: 0
# num_layers: 6
# units_0: 111
# units_1: 98
# units_2: 65
# learning_rate: 0.05
# beta_1: 0.7
# beta_2: 0.99
# optimizer: 6
# units_3: 26
# units_4: 111
# units_5: 91
# Score: 20.706668853759766

# Trial 12 summary
# Hyperparameters:
# l1_lr: 1e-06
# l2_lr: 0.0001
# bias_lr: 0.001
# activation: 10
# init: 1
# num_layers: 4
# units_0: 39
# units_1: 62
# units_2: 102
# learning_rate: 0.05
# beta_1: 0.7
# beta_2: 0.7
# optimizer: 0
# units_3: 51
# units_4: 83
# Score: 25.305789947509766

# Trial 11 summary
# Hyperparameters:
# l1_lr: 0.001
# l2_lr: 0.0
# bias_lr: 0.001
# activation: 1
# init: 1
# num_layers: 3
# units_0: 11
# units_1: 89
# units_2: 115
# learning_rate: 0.01
# beta_1: 0.7
# beta_2: 0.7
# optimizer: 0
# units_3: 46
# units_4: 47
# Score: 26.025346755981445

# Trial 07 summary
# Hyperparameters:
# l1_lr: 0.0001
# l2_lr: 0.001
# bias_lr: 0.001
# activation: 1
# init: 0
# num_layers: 4
# units_0: 41
# units_1: 115
# units_2: 72
# learning_rate: 0.02
# beta_1: 0.99
# beta_2: 0.99
# optimizer: 6
# units_3: 95
# units_4: 38
# Score: 27.15325927734375